In [1]:
from pathlib import Path
import pandas as pd
import os

r = Path('/mnt/hpc/pathology/clam_features/primary_vs_metastasis')

dmtr = pd.read_csv(r.parent.parent.parent / 'rens' / 'tables' / 'dmtr.csv').set_index('id')
prim_vs_met = pd.read_csv('/mnt/hpc/rens/CLAM/dataset_csv/primary_vs_metastasis.csv').set_index('slide_id')


/tmp/ipykernel_24306/508218498.py:7: DtypeWarning: Columns (27,29,175,234,249,259,275,281,313,318,351,393,440,538,547,550,578,585,591,654,660,662,663,664,666,667,669,670,671,676,677,679,680,681,682,685,794,797,805,808,809,818,819,821,822,823,828,851,852,854,855,856,863,864,866,867,868,869,870,872,873,874,875,876,878,879,880,881,882,884,885,886,899,900,902,903,904,905,906,908,909,910,911,912,914,915,916,917,918,920,921,922,924) have mixed types. Specify dtype option on import or set low_memory=False.
  dmtr = pd.read_csv(r.parent.parent.parent / 'rens' / 'tables' / 'dmtr.csv').set_index('id')


In [2]:
case_id = []
slide_id = []
response = []
metastasis = []
# for root, _, files in os.walk(r / 'primary'):
#     pt_files = [f for f in files if f.endswith('.pt')]
#     if pt_files:
#         slide_id.extend([f[:-3]
#         primary.extend(['primary'] * len(pt_files))
#         if 'maxima' in root:
#             case_id.extend([f.split(' ')[0].replace('-','_')
#         if 'umcu' in root:
#             case_id.extend(['_'.join(f.replace('-','_').replace(' ','_').split('_')[:2])
#         if 'radboud' in root:
#             case_id.extend([f[:11].replace('-','_')
#         if 'zuyderland' in root:
#             case_id.extend([f[:11].replace('-','_')
#         if 'lumc' in root:
#             case_id.extend(['PREM' + f.split('PREM')[1][:7].replace('-','_')
#         if 'vumc' in root:
#             case_id.extend([f[:11]
#         if 'mst' in root:
#             case_id.extend([f[:11].replace('-','_')
#         if 'isala' in root:
#             case_id.extend([f[:11].replace('-','_')
#         if 'amphia' in root:
#             case_id.extend([f[:11].replace('-','_')

for root, _, files in os.walk(r):
    pt_files = [f for f in files if f.endswith('.pt')]
    for file in pt_files:

        slide_id.append(file[:-3])
        if 'MAX-' in file:
            patient = file.split(' ')[0].replace('-','_').replace('_9hy6vdr5tygrr','')
        if 'UNI-' in file or 'IM' in file or 'M' in file:
            patient = '_'.join(file.replace('-','_').replace(' ','_').split('_')[:2])
        if 'PREM-RA-' in file:
            patient = file[:11].replace('-','_')
        if 'PREM_ZU_' in file:
            patient = file[:11].replace('-','_')
        if 'PREM-LU-' in file:
            patient = 'PREM' + file.split('PREM')[1][:7].replace('-','_')
        if 'PREM_VU_' in file:
            patient = file[:11]
        if 'PREM-MS-' in file:
            patient = file[:11].replace('-','_')
        if 'PREM-IS-' in file:
            patient = file[:11].replace('-','_')
        if 'PREM-AM-' in file:
            patient = file[:11].replace('-','_')

        try:
            metastasis.append(prim_vs_met.loc[file[:-3], 'primary'] == 'metastasis')   
        except:
            metastasis.append(True)
        
        case_id.append(patient)
        try:
            response.append(dmtr.loc[patient, 'response'])
        except Exception as e:
            response.append(float('nan'))

df = pd.DataFrame(data={
    'case_id':case_id,
    'slide_id':slide_id,
    'response': response,
    'metastasis':metastasis
})

metastasis_only = df[df.metastasis].drop(columns=['metastasis']).replace({'response':{0:'no_response',1:'response'}})
metastasis_only = metastasis_only.dropna()

bad = [
    'PREM-AM-003__ - 2021-11-04 11.35.09',
    'PREM-AM-003__ - 2021-11-04 11.35.09',
    'PREM-AM-013__ - 2021-11-04 11.16.34',
    'PREM-AM-029__ - 2021-11-04 09.40.15',
    'PREM-AM-029__II - 2021-11-04 09.46.16',
    'PREM-AM-048__ - 2021-11-04 10.44.34',
    'PREM-AM-052__ - 2021-11-04 10.35.01',
    'PREM-AM-084__ - 2021-11-04 12.40.30',
    'PREM-AM-094__ - 2021-11-04 12.22.41',
    'PREM-AM-112__ - 2021-11-04 12.01.25',
    '2A_HE MD_PREM-LU-416 - 2021-06-16 10.00.47',
    'MAX-060 259615432a - 2022-09-29 12.44.28',
    'PREM-RA-017_1ca56bd887 HE1',
    'IM_046_8af4fb5881-I1_HE  1-001',
    'IM_161_4aae1a3640-I1_HE  1-001',
    'IM_161_4aae1a3640-I1_HE  1-003',
    'UNI-027_7bcf0a9271-I1_HE  1-002',
    'PREM_ZU_002 - c16-53153 - 2021-08-12 15.18.13',
    'PREM_ZU_023 - C18-001050-I-4 - 2021-08-13 11.00.42',
    'PREM_ZU_023 - C18-001050-I-5 - 2021-08-13 11.01.25',
    'PREM_ZU_023 - C18-001050-I-5 - 2021-08-13 11.01.50',
    'PREM_ZU_023 - C18-001050-I-A-1 - 2021-08-13 11.02.20',

    'PREM_ZU_002 - c16-53153 - 2021-08-12 15.27.59',
    'PREM_ZU_002 - C16-53153 - 2021-08-12 15.24.39',
    'PREM_ZU_020 - C16-50968 - 2021-08-13 10.59.56',
    'PREM_ZU_030 - T19-000029-I-A-4 - 2021-08-12 15.34.38',
    'PREM_ZU_020 - C16-50968 - 2021-08-13 10.58.33',
    'PREM_ZU_002 - c16-53153 - 2021-08-12 15.25.24',
    'PREM_ZU_020 - C16-50968 - 2021-08-13 10.57.01'
]

metastasis_only = metastasis_only[~(metastasis_only.slide_id.isin(bad))]

In [24]:
metastasis_only.to_csv('/mnt/hpc/rens/CLAM/dataset_csv/metastasis_response.csv', index=False)

In [11]:
previous = pd.read_csv('/mnt/hpc/rens/CLAM/dataset_csv/metastasis_response.csv')

In [14]:
new_slides = [slide for slide in metastasis_only.slide_id.values if not slide in previous.slide_id.values]

In [17]:
[s for s in new_slides if not ('UNI' in s or 'PREM-RA' in s or 'IM' in s or 'M-' in s)]

['MAX-021 32fd721f35 - 2022-09-29 12.01.19',
 'MAX-012_9hy6vdr5tygrr - 2022-11-10 13.17.48',
 'PREM_ZU_002 - c16-53153 - 2021-08-12 15.27.59',
 'PREM_ZU_002 - C16-53153 - 2021-08-12 15.24.39',
 'PREM_ZU_020 - C16-50968 - 2021-08-13 10.59.56',
 'PREM_ZU_030 - T19-000029-I-A-4 - 2021-08-12 15.34.38',
 'MAX-063 9683d525ce - 2022-09-29 11.58.41',
 'MAX-089 6ee5b4e028 - 2022-09-29 12.03.52',
 'PREM_ZU_020 - C16-50968 - 2021-08-13 10.58.33',
 'MAX-111 4e227ee815 - 2022-09-29 11.41.24',
 'PREM_VU_160 58ef7bf546 - 2022-06-14 09.46.28',
 'PREM_ZU_002 - c16-53153 - 2021-08-12 15.25.24',
 'MAX-032 a168bdef25 - 2022-09-29 11.51.23',
 'PREM_ZU_020 - C16-50968 - 2021-08-13 10.57.01']

In [82]:
len(case_id)

1488

In [4]:
df.to_csv('/mnt/hpc/rens/CLAM/dataset_csv/primary_vs_metastasis.csv', index=False)

In [7]:
df.replace({'primary':{1:'primary',0:'metastasis'}}).to_csv('/mnt/hpc/rens/CLAM/dataset_csv/primary_vs_metastasis.csv', index=False)

In [52]:
import shutil

target = r / 'primary_vs_metastasis'
for part in list(r.iterdir())[:-1]:
    for center in part.iterdir():
        for f in (center / 'h5_files').iterdir():
            shutil.move(
                f,
                target / 'h5_files' / f.name
            )
        for f in (center / 'pt_files').iterdir():
            shutil.move(
                f,
                target / 'pt_files' / f.name
            )

In [156]:
metastasis_only = pd.read_csv('/mnt/hpc/rens/CLAM/dataset_csv/metastasis_response.csv')

In [164]:
metastasis_only

,case_id,slide_id,response
0,PREM_ZU_038,PREM_ZU_038 - T20-020312-I-S-1 - 2021-08-12 15...,no_response
1,PREM_VU_102,PREM_VU_102 71d95b93cb - 2022-06-14 11.49.03,response
2,PREM_ZU_036,PREM_ZU_036 - T17-5592534 - 2021-08-13 14.03.27,no_response
3,MAX_135,MAX-135 7c594aacd2 - 2022-09-29 11.53.53,no_response
4,UNI_029,IM_030_d405a26bdd-I1_HE 2-003,no_response
...,...,...,...
708,PREM_MS_289,PREM-MS-289_7596e51925 HE1 Metastasegq - 2022-...,response
709,PREM_MS_289,IM_015_678b3f808f-I1_HE 1-002,response
710,PREM_VU_191,PREM_VU_191 597d4ae2e7 - 2022-06-14 11.46.56,no_response
711,PREM_LU_561,1A__PREM-LU-561 - 2021-06-15 16.32.37,no_response


In [192]:
split0 = pd.read_csv('/mnt/hpc/rens/CLAM/splits/task_4_metastasis_response_100/splits_0_bool.csv').set_index('Unnamed: 0')

In [193]:
split0

,train,val,test
Unnamed: 0,,,
IM_004_ff63eb3c79-II7_HE 1-025,True,False,False
IM_004_ff63eb3c79-I5_HE 1-019,True,False,False
IM_004_ff63eb3c79-I5_HE 2-023,True,False,False
IM_004_ff63eb3c79-VII3_HE 1-013,True,False,False
IM_004_ff63eb3c79-III2_HE 1-040,True,False,False
...,...,...,...
PREM-IS-074_fab78c9efd II1 HE1 - 2022-03-24 17.39.37,False,False,True
IM_085_2847946502-I4_HE 1-003,False,False,True
IM_085_2847946502-I2_HE 1-001,False,False,True


In [194]:
patient = []
for index in split0.index:
    patient.append(metastasis_only.set_index('slide_id').loc[index,'case_id'])

split0['case_id'] = patient

In [195]:
train = split0[split0.train].case_id.unique()
val = split0[split0.val].case_id.unique()
test = split0[split0.test].case_id.unique()

In [197]:
[pt for pt in train if pt in val]

[]

In [199]:
split0

,train,val,test,case_id
Unnamed: 0,,,,
IM_004_ff63eb3c79-II7_HE 1-025,True,False,False,IM_004
IM_004_ff63eb3c79-I5_HE 1-019,True,False,False,IM_004
IM_004_ff63eb3c79-I5_HE 2-023,True,False,False,IM_004
IM_004_ff63eb3c79-VII3_HE 1-013,True,False,False,IM_004
IM_004_ff63eb3c79-III2_HE 1-040,True,False,False,IM_004
...,...,...,...,...
PREM-IS-074_fab78c9efd II1 HE1 - 2022-03-24 17.39.37,False,False,True,PREM_IS_074
IM_085_2847946502-I4_HE 1-003,False,False,True,IM_085
IM_085_2847946502-I2_HE 1-001,False,False,True,IM_085


In [144]:
713 * 0.75

534.75